# Brain Masks Analysis

Brain Masks Analysis

In [1]:
import nibabel as nib
from glob import glob
from tqdm.notebook import tqdm
import os
import pandas as pd
from p_tqdm import p_map
from nibabel import processing
import matplotlib.pyplot as plt

freesurfer_folder: Literal['../data/raw/FREESURFER/'] = "../data/raw/FREESURFER/"

NameError: name 'Literal' is not defined

We downloaded the MRI and Freesurfer files.
For each MRI session we select the T1w (if there are more "run" in the same session we select the first),
the MR scan (and the corresponding brain mask)

In [ ]:
brainmask_list = glob(freesurfer_folder+"*/*/brainmask.mgz", recursive=True)
print(f"Found {len(brainmask_list)} brainmask")

Found 0 brainmask


In [ ]:

def get_info(file):
    result = dict()
    scan = nib.load(file)
    result['file_path'] = file
    result["src_orient"] = ''.join(nib.aff2axcodes(scan.affine))
    for i, v in enumerate(scan.header.get_data_shape()):
        result[f"src_shape_{i}"] = v
    for i, v in enumerate(scan.header.get_zooms()):
        result[f"src_res_{i}"] = v
    for i, v in enumerate(x.max()-x.min() for x in scan.get_fdata().nonzero()):
        result[f"src_brain_{i}"] = v
    result['srcVolume'] = scan.get_fdata().count_nonzero()
    scan = processing.conform(scan, voxel_size=(1.0,1.0,1.0), orientation='LPS')

    result["dst_orient"] = ''.join(nib.aff2axcodes(scan.affine))
    for i, v in enumerate(scan.header.get_data_shape()):
        result[f"dst_shape_{i}"] = v
    for i, v in enumerate(scan.header.get_zooms()):
        result[f"dst_res_{i}"] = v
    for i, v in enumerate(x.max()-x.min() for x in scan.get_fdata().nonzero()):
        result[f"dst_brain_{i}"] = v
    return result

pd.DataFrame([get_info(brainmask_list[0]),get_info(brainmask_list[1])])

IndexError: list index out of range

In [ ]:
try:
    mask_data = pd.read_csv("../data/interim/maskdata.csv")
    print("Data of brain masks already exists :)")
except FileNotFoundError:
    print("No cached brain mask data i'm going to create one :(")
    temp = p_map(get_info, brainmask_list, desc="Analyzin brain masks")
    mask_data = pd.DataFrame(temp)
    mask_data.to_csv("../data/interim/maskdata.csv")

In [ ]:
mask_data[["dst_brain_0", "dst_brain_1","dst_brain_2"]].describe()

In [ ]:
mask_data.sort_values("dst_brain_1", ascending=False)

In [ ]:
data = processing.conform(nib.load("../data/raw/FREESURFER/OAS30244_MR_d0104/mri/brainmask.mgz"), voxel_size=(1.0,1.0,1.0), orientation='LIA').get_fdata()

plt.imshow(data[:,150,...])

The brain mask, available in the freesurfer files, is binarized with a zero threshold, and a morphological closing is performed on the mask (to remove possible gaps within the volume). Finally, by point wise multiplication, we extract the masked volume that represent our ROI.